In [49]:
import netCDF4 as nc
import numpy as np
import json
import math

In [75]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [17]:
fn1 = 'sauce/Wind10.nc4'
fn2 = 'sauce/Wind11.nc4'

ds1 = nc.Dataset(fn1
ds2 = nc.Dataset(fn2)

# Life is pain

In [170]:
U_GRD_L103 = ds1["U_GRD_L103"][:]
V_GRD_L103 = ds1["V_GRD_L103"][:]
lat = ds1["lat"][:]
lon = ds1["lon"][:]
U_GRD_L103

masked_array(
  data=[[[[ 5.750348  ,  5.750348  ,  5.750348  , ...,  5.6503477 ,
            5.6503477 ,  5.6503477 ],
          [ 4.350348  ,  4.350348  ,  4.350348  , ...,  4.350348  ,
            4.350348  ,  4.350348  ],
          [ 3.950348  ,  3.950348  ,  3.950348  , ...,  3.850348  ,
            3.950348  ,  3.950348  ],
          ...,
          [-3.2496521 , -3.149652  , -3.149652  , ..., -3.349652  ,
           -3.349652  , -3.2496521 ],
          [-1.6496521 , -1.5496521 , -1.5496521 , ..., -1.7496521 ,
           -1.6496521 , -1.6496521 ],
          [-0.1496521 , -0.0496521 , -0.0496521 , ..., -0.2496521 ,
           -0.2496521 , -0.1496521 ]],

         [[ 5.61499   ,  5.6249905 ,  5.63499   , ...,  5.58499   ,
            5.5949903 ,  5.60499   ],
          [ 4.2449903 ,  4.2449903 ,  4.2449903 , ...,  4.2449903 ,
            4.2449903 ,  4.2449903 ],
          [ 3.6949902 ,  3.6949902 ,  3.7149901 , ...,  3.6949902 ,
            3.6949902 ,  3.6949902 ],
          ...,


In [8]:
rad_fn = 'sauce/Sac1.nc4'
rad_ds = nc.Dataset(rad_fn)

In [19]:
lst = []
n = 100
# Accuracy of meuserment. 1 is the maxium and most refined, but it will take 49 times longer to compute

for ix, x in enumerate(lat):
    if not ix % n:
        for iy, y in enumerate(lon):
            if not iy % n:

                V = math.sqrt(ds1["U_GRD_L103"]['0'][6][ix][iy] ** 2 + ds1["V_GRD_L103"]['0'][6][ix][iy] ** 2)
                mss_mod = 0
        
                if V <= 3.49:
                    mss_mod = 0.0035 * (V + 0.62)
                else:
                    mss_mod = 0.0035 * (6*math.log(V) - 3.39)

                lst.append((y, x, mss_mod))
                
    


In [141]:
V = np.sqrt(ds1["U_GRD_L103"]['0'][6] * ds1["U_GRD_L103"]['0'][6]  + ds1["U_GRD_L103"]['0'][6] * ds1["U_GRD_L103"]['0'][6])
cond = V <= 3.49
mss_mod_db = np.where((V <= 3.49), 0.0035 * (V + 0.62),  0.0035 * (6*np.log(V) - 3.39))

 

In [146]:
radds_lat = np.array(rad_ds["lat"][:])
radds_lon = np.array(rad_ds["lon"][:])

/var/folders/0g/k0tsczj93vq5fwg_pnt31zv40000gn/T/ipykernel_46745/898792714.py:2: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  radds_lon = np.array(rad_ds["lon"][:])


In [178]:
%%cython
from __main__ import radds_lat, radds_lon, lat, lon
import numpy
cimport numpy 




cdef float rad_var
cdef numpy.ndarray arr
cdef int arr_shape = arr.shape[0]


 
cdef radtomod(var, mod):
    for rad_var in numpy.nditer(var, op_flags = ['readwrite']):
        radtomod_ivar =  numpy.abs(mod - rad_var).argmin()
        rad_var[...] = mod[radtomod_ivar]
    return var

radds_lat = radtomod(radds_lat, lat)
radds_lon = radtomod(radds_lon, lon)



In [152]:
# print(mss_mod_db.shape, lon.shape, lat.shape)
print(radds_lat)

[-32.75 -26.75 -35.5  ... -26.75 -23.5  -31.75]


In [200]:
lst = np.array(lst)
mod_x = 0
fin_db = []

for rad_lat, rad_lon, mss_rad in zip(radds_lat, radds_lon, rad_ds["mean_square_slope"][:]):

        mod_i_lat = np.nonzero(lat == rad_lat)
        mod_i_lon = np.nonzero(lon == rad_lon)
        
        
        if not isinstance(mss_rad, np.float32):
                continue

        if V[mod_i_lat, mod_i_lon] < 3 or V[mod_i_lat, mod_i_lon] > 11:
                # print("Wrong V: ", V[mod_i_lat, mod_i_lon])
                continue
        mss_mod = mss_mod_db[mod_i_lat, mod_i_lon]
        mss_anom = (mss_rad / mss_mod) - 1
        
        dens = 2035*math.exp(-23.18*mss_anom)
        
        fin_db.append((rad_lat, rad_lon, dens))
        
        

/var/folders/0g/k0tsczj93vq5fwg_pnt31zv40000gn/T/ipykernel_46745/3488110762.py:5: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  for rad_lat, rad_lon, mss_rad in zip(radds_lat, radds_lon, rad_ds["mean_square_slope"][:]):


In [197]:
rad_lat = 3
rad_lon = -59.5

rad_lon += 180
rad_stack = np.stack((radds_lat, radds_lon), axis=1)

mod_i_lat = np.nonzero(lat == rad_lat)
mod_i_lon = np.nonzero(lon == rad_lon)

mod_i_rad = np.where((rad_stack == (rad_lat, rad_lon)).all(axis = 1))

# print(mss_i_rad_lat, mss_i_rad_lon)

# print(radds_lat[1467225])


# print(mod_i_lat, mod_i_lon)

# if not isinstance(mss_rad, np.float32):
#         continue

if V[mod_i_lat, mod_i_lon] < 3 or V[mod_i_lat, mod_i_lon] > 11:
        print("Wrong V: ", V[mod_i_lat, mod_i_lon])
        

mss_rad = rad_ds["mean_square_slope"][mod_i_rad]
mss_mod = mss_mod_db[mod_i_lat, mod_i_lon]
mss_anom = (mss_rad / mss_mod) - 1


dens = 2035*np.exp(-23.18*mss_anom)

print("mod_i_rad: ", mod_i_rad)
print("V: ", V[mod_i_lat, mod_i_lon])
print('mss_rad: ', mss_rad, ' mss_mod: ', mss_mod, ' mss_anom: ', mss_anom)
print("Desnisty: ", dens)

print(dens)

mod_i_rad:  (array([1025093]),)
V:  [[4.223948001861572]]
mss_rad:  [0.00645592]  mss_mod:  [[0.01839118]]  mss_anom:  [[-0.6489665508270264]]
Desnisty:  [[6945057792.0]]
[[6945057792.0]]


/var/folders/0g/k0tsczj93vq5fwg_pnt31zv40000gn/T/ipykernel_46745/673374498.py:26: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  mss_rad = rad_ds["mean_square_slope"][mod_i_rad]


In [174]:
dens

nan

In [22]:
# lst = np.array(lst)
# mod_x = 0
# fin_db = []

# for x, y, mss_rad in zip(rad_ds["lat"][:], rad_ds["lon"][:], rad_ds["mean_square_slope"][:]):
#         if not isinstance(mss_rad, np.float32):
#                 continue

#         modi_x = np.abs(lst[:, 1] - x).argmin()
#         modi_y = np.abs(lst[:, 0] - y).argmin()

#         arr = lst[:, 0:2]
#         mod_y = lst[modi_y, 0]
#         mod_x = lst[modi_x, 1]
#         mod_i = np.where((arr == (mod_y, mod_x)).all(axis = 1))

#         mss_mod = lst[mod_i[0][0], 2]
        
#         mss_anom = (mss_rad / mss_mod) - 1
#         dens = 2035*math.exp(-23.18*mss_anom)
#         fin_db.append((lst[modi_y, 0], lst[modi_x, 1], dens))
        
        

/var/folders/0g/k0tsczj93vq5fwg_pnt31zv40000gn/T/ipykernel_46745/992665060.py:5: UserWarning: WARNING: valid_range not used since it
cannot be safely cast to variable data type
  for x, y, mss_rad in zip(rad_ds["lat"][:], rad_ds["lon"][:], rad_ds["mean_square_slope"][:]):


In [166]:
json_db = []
for lat, lon, dens in fin_db:
  stg = {
        "type": "Feature",
        "properties": {
            "conScale": float(dens)
        },
        "geometry": {
          "type": "Point",
          "coordinates": [
            float(lat),
            float(lon - 180)
            
          ]
        }
    }
  json_db.append(stg)
json_str = {
  "type": "FeatureCollection",
  "features" : json_db 
}
with open("destination/data_base_large.json", "w") as write_file:
    json.dump(json_db, write_file, indent=4)



In [ ]:
lst = np.array(lst)
print(rad_ds["mean_square_slope"][:])
